In [408]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [409]:
class Value:
    def __init__(self, data, _children=(), _op='', label="") -> None:
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = _children
        self._op = _op
        self.label = label
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    def __radd__(self, other):
        return self + other
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):
        return self * other
    def __truediv__(self, other):
        return self * other**-1
    def __neg__(self):
        return self * -1
    def __sub__(self, other):
        return self + (-other)
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')
        def _backward():
            self.grad += other * (self.data**(other-1)) * out.grad
        out._backward = _backward
        return out
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    def __repr__(self) -> str:
        return f'Value(data={self.data},label={self.label})'

In [410]:
import random


class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [411]:
x = [1, 2, 3]
n = Neuron(3)
n(x)

Value(data=-0.992425936662638,label=)

In [412]:
x = [1, 2, 3]
n = Layer(3, 4)
n(x)

[Value(data=-0.9987024496137216,label=),
 Value(data=-0.9546893578954241,label=),
 Value(data=0.9813795056634552,label=),
 Value(data=0.9632288953968734,label=)]

In [413]:
x = [1, 2, 3]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=-0.5643797254829152,label=)

In [414]:
n = MLP(3, [4, 4, 1])
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [415]:
for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  # mean squared error
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)

0 6.457154720208456
1 3.6328293427364446
2 2.514796924217231
3 1.871884653628027
4 0.20635766848094428
5 0.10607963659852836
6 0.08215954594550964
7 0.06745904394824609
8 0.05725290788981467
9 0.04970565538381194
10 0.043887842754215894
11 0.039264584464667254
12 0.03550278710252234
13 0.032383175064212975
14 0.029755178072166814
15 0.02751186905651156
16 0.025575185427318164
17 0.0238867961954147
18 0.02240223823782237
19 0.02108702910048613


In [416]:
ypred

[Value(data=0.9399567210329848,label=),
 Value(data=-0.9378569079950722,label=),
 Value(data=-0.9294674890916282,label=),
 Value(data=0.9070202453627522,label=)]